In [7]:
import glob     # Import glob to easily loop over files
import pympi    # Import pympi to work with elan files
import nltk     # Import nltk to use tokenizer
import pandas as pd
import shutil
import os

In [8]:
def correctionSearch(notExNode, corrections, all_node_names, foundExistingNodes, file, role, interval, warninglog, initNode):
    nodeUnknown = True
    for correction in corrections:
        if correction(notExNode).replace("X","") in all_node_names:
            foundExistingNodes.append(correction(notExNode))
            warninglog = warninglog+"Node renamed: "+initNode+" -> "+correction(notExNode)+"; File: "+file+"; Role: "+role+"; Timeslot"+interval+"\n"
            print(warninglog)
            return False, warninglog
    if nodeUnknown:
        for corIdx in range(len(corrections)):
            correction = corrections[corIdx]
            ncorrections = corrections.copy()
            del ncorrections[corIdx]
            broadSearchRes = correctionSearch(correction(notExNode), ncorrections, all_node_names, foundExistingNodes, file, role, interval, warninglog, initNode)
            warninglog = broadSearchRes[1]
            nodeUnknown = nodeUnknown and broadSearchRes[0]
            if not nodeUnknown:
                return False, warninglog
    return nodeUnknown, warninglog
        

In [9]:
def correctionPoint(annotation):
    prepend =""
    if "X" in annotation:
        prepend="X"
        annotation = annotation.replace("X","")
    return annotation +  "."+prepend

def correctionXSpace(annotation):
    if "X" in annotation:
        return annotation.replace("X","").strip()+"X"
    elif "x" in annotation:
        return annotation.replace("x","").strip()+"X"
    else:
        return annotation

def correctionSpelling(token):
    prepend =""
    if "X" in token:
        prepend="X"
        token = token.replace("X","")
    if token == "1.0":
        token = "1"
    elif token == "1.0.":
        token = "1"
    elif token == "1.":
        token = "1"
    elif token == "8.2.1":
        token = "8.2"
    elif token == "5.7.":
        token = "6.2.2."
    return token+prepend

In [10]:
def correctNodeNames(annotation, all_node_names, errorlog, warninglog, file, role, interval, eaf):
    annotationText = annotation[2]
    foundNodes = annotationText.split(",")
    foundNodes = list(map(lambda x: x.strip(),foundNodes))
    foundNotExistingNodes = list(filter(lambda x: (not x in all_node_names and not x.replace("X", "").strip() in all_node_names) and not x == "",foundNodes)) 
    foundExistingNodes = list(filter(lambda x: x in all_node_names or x.replace("X", "").strip() in all_node_names,foundNodes))
    # correctionXSpace must be the last one !!!!!
    corrections = [correctionXSpace, correctionPoint, correctionSpelling]

    for notExNode in foundNotExistingNodes:
        correctionUnsuccessful, warninglog = correctionSearch(notExNode, corrections, all_node_names, foundExistingNodes, file, role, interval, warninglog, notExNode)    
        if correctionUnsuccessful:
            errorlog = errorlog+"Node unknown: "+notExNode+"; File: "+file+"; Role: "+role+"; Timeslot"+interval+"\n"
    if len(foundNotExistingNodes) > 0: 
        eaf.remove_annotation(role+"_Gamenodes", annotation[0])
        eaf.add_annotation(role+"_Gamenodes",annotation[0],annotation[1]-1 ,str(",".join(foundExistingNodes)))

    #print("Change annotation "+annotation[2]+interval+" to "+ str(",".join(foundExistingNodes)))
    return errorlog, warninglog


    

In [11]:
def ms2human_readable(ms):
    mss = ms % 1000
    sec = int(ms / 1000)
    min = int(sec/60)
    sec = sec - min*60
    h = int(min/60)
    min = min - h*60
    return str(h)+":"+str(min)+":"+str(sec)+" "+str(mss)

In [12]:
keys = pd.read_csv("Nodeaufstellung.csv")["NodeID"].to_list()
shutil.rmtree(os.getcwd()+'\\out')
os.mkdir("out")
files = glob.glob("in/*.eaf")
errorlog = ""
warninglog = ""
for file in files:
    eafob = pympi.Elan.Eaf(file)
    singleTextEX = "( EX "+file+")"
    singleTextEE = "( EE "+file+")"
    for annotation in eafob.get_annotation_data_for_tier("EX_Gamenodes"):
        errorlog, warninglog = correctNodeNames(annotation,keys, errorlog, warninglog, file, "EX", "["+str(ms2human_readable(annotation[0]))+";"+str(ms2human_readable(annotation[1]))+"]",eafob)
    for annotation in eafob.get_annotation_data_for_tier("EE_Gamenodes"):
        errorlog, warninglog = correctNodeNames(annotation,keys, errorlog, warninglog, file, "EE", "["+str(ms2human_readable(annotation[0]))+";"+str(ms2human_readable(annotation[1]))+"]",eafob)
    oldFileNameSplit = file.split("\\")[1].split(".")
    pympi.Elan.to_eaf("out/"+oldFileNameSplit[0]+"_changed."+oldFileNameSplit[1],eafob)
with open("out/error.log","w") as f:
    f.write(errorlog)
with open("out/warning.log","w") as f:
    f.write(warninglog)

Node renamed: 5.4.2 -> 5.4.2.; File: in\VP23_P2_A01.eaf; Role: EX; Timeslot[0:4:42 721;0:4:51 121]

Node renamed: 5.4.2 -> 5.4.2.; File: in\VP23_P2_A01.eaf; Role: EX; Timeslot[0:4:42 721;0:4:51 121]
Node renamed: 1.0. -> 1; File: in\VP26_P2_A01.eaf; Role: EX; Timeslot[0:0:5 240;0:0:8 399]

Node renamed: 5.4.2 -> 5.4.2.; File: in\VP23_P2_A01.eaf; Role: EX; Timeslot[0:4:42 721;0:4:51 121]
Node renamed: 1.0. -> 1; File: in\VP26_P2_A01.eaf; Role: EX; Timeslot[0:0:5 240;0:0:8 399]
Node renamed: 1.0. -> 1; File: in\VP27_P2_A01.eaf; Role: EX; Timeslot[0:0:6 670;0:0:8 789]

Node renamed: 5.4.2 -> 5.4.2.; File: in\VP23_P2_A01.eaf; Role: EX; Timeslot[0:4:42 721;0:4:51 121]
Node renamed: 1.0. -> 1; File: in\VP26_P2_A01.eaf; Role: EX; Timeslot[0:0:5 240;0:0:8 399]
Node renamed: 1.0. -> 1; File: in\VP27_P2_A01.eaf; Role: EX; Timeslot[0:0:6 670;0:0:8 789]
Node renamed: 1.0. -> 1; File: in\VP29_P2_A01.eaf; Role: EX; Timeslot[0:0:2 830;0:0:5 839]

Node renamed: 5.4.2 -> 5.4.2.; File: in\VP23_P2_A01.ea